# Castaways reactions
> How did contestants respond after their torch was snuffed? This script fetches the growing log and scores the acknowlegement of his/her tribe.rt    

#### Load Python tools and Jupyter config

In [1]:
import os
import json
import pandas as pd
import jupyter_black
import gspread
from google.oauth2.service_account import Credentials

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [3]:
# Use the environment variable to Google account key
credentials_path = os.getenv("GOOGLE_SHEETS")

In [4]:
# Output paths
csv_output_path = "../data/processed/survivor_vote_off_reactions.csv"
json_output_path = "../data/processed/survivor_vote_off_reactions.json"

#### Google Sheets creds

In [5]:
# Load the client secrets file
service_account_file = "/Users/mstiles/.google_service_account.json"

In [6]:
# Define the scope
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# Authenticate using the service account file
credentials = Credentials.from_service_account_file(service_account_file, scopes=scope)

# Authorize the gspread client
client = gspread.authorize(credentials)

---

## Read data

#### Get data entry table

In [7]:
# Open the Google Spreadsheet by name
spreadsheet_name = "survivor_vote_offs"
spreadsheet = client.open(spreadsheet_name).get_worksheet(0)

# Fetch data from the worksheet
data = spreadsheet.get_all_records()

In [8]:
data_entry = pd.DataFrame(data)

In [9]:
# clean castaways table
vote_offs_lookup = pd.read_csv("../data/processed/survivor_voteoff_order.csv")

In [10]:
# Assertion to ensure the lengths are the same
assert len(data_entry) == len(
    vote_offs_lookup
), f"Assertion failed: Lengths do not match (data_entry: {len(data_entry)}, vote_offs_lookup: {len(vote_offs_lookup)}"

print("Lengths match successfully!")

Lengths match successfully!


In [11]:
# Merge them together to add ids to Google Sheet
merged = pd.merge(
    data_entry, vote_offs_lookup, on=["season", "vote"], how="right", indicator=True
)

In [12]:
merged["match"] = merged["castaway"] == merged["voted_out"]

In [13]:
assert merged[
    "match"
].all(), "Assertion failed: There are mismatched rows in the DataFrame."

print("All rows matched successfully!")

All rows matched successfully!


In [14]:
# List of columns to convert
bool_columns = ["acknowledge", "ack_gesture", "ack_speak", "ack_look", "ack_smile"]

# Convert the columns from string to boolean
for col in bool_columns:
    merged[col] = merged[col].map({"TRUE": True, "FALSE": False})

In [15]:
# Define acknowledgment columns
ack_columns = ["ack_gesture", "ack_speak", "ack_look", "ack_smile"]

In [16]:
merged.sample(100).acknowledge.value_counts()

acknowledge
True     55
False    40
Name: count, dtype: int64

In [17]:
# Calculate acknowledgment score as the count of True values in the acknowledgment columns
merged["ack_score"] = merged[ack_columns].sum(axis=1)

In [18]:
# Save to CSV
merged.to_csv(csv_output_path, index=False)

In [19]:
# Save to JSON
merged.to_json(json_output_path, orient="records", indent=4)

----

#### How much is left? 

In [20]:
left = len(data_entry.query("acknowledge == ''"))
done = len(data_entry.query("acknowledge != ''"))
total = len(data_entry)

In [21]:
round((done / total) * 100, 2)

94.08